In [33]:
import numpy as np

In [2]:
##training data

X = np.array([
    [2,1,0,0,0,0],
    [2,0,1,0,0,0],
    [1,0,0,1,0,0],
    [1,0,0,0,1,1]
])
y = np.array([0,0,0,1])

In [3]:
np.unique(y)

array([0, 1])

In [7]:
separated = []
for c in np.unique(y): #loop through each class
    inner = []
    for x, t in zip(X,y): #loop through each record
        if t == c:
            inner.append(x)
    separated.append(inner)
print (separated)

[[array([2, 1, 0, 0, 0, 0]), array([2, 0, 1, 0, 0, 0]), array([1, 0, 0, 1, 0, 0])], [array([1, 0, 0, 0, 1, 1])]]


In [8]:
separated = [[x for x, t in zip(X, y) if t == c] for c in np.unique(y)]
print (separated)

[[array([2, 1, 0, 0, 0, 0]), array([2, 0, 1, 0, 0, 0]), array([1, 0, 0, 1, 0, 0])], [array([1, 0, 0, 0, 1, 1])]]


In [11]:
X.shape

(4, 6)

In [12]:
count_samples = X.shape[0] #number of records

In [17]:
separated = [[x for x,t in zip(X,y) if t == c] for c in np.unique(y)]

In [38]:
class_log_prior_ = [np.log(len(all_samples_in_class)/count_samples) for all_samples_in_class in separated] #priors
#list of priors

In [39]:
class_log_prior_

[-0.2876820724517809, -1.3862943611198906]

In [57]:
count = []
for all_samples_in_class in separated:
    print(np.array(all_samples_in_class).sum(axis=0)) #turn list into numpy then take the sum along each column aka word
count = np.array([np.array(all_samples_in_class).sum(axis=0) for all_samples_in_class in separated])


[5 1 1 1 0 0]
[1 0 0 0 1 1]


In [58]:
print(count)

[[5 1 1 1 0 0]
 [1 0 0 0 1 1]]


In [72]:
count.sum(axis=1)[np.newaxis].T

array([[8],
       [3]])

In [67]:
count/count.sum(axis=1)[np.newaxis].T

array([[0.625     , 0.125     , 0.125     , 0.125     , 0.        ,
        0.        ],
       [0.33333333, 0.        , 0.        , 0.        , 0.33333333,
        0.33333333]])

In [ ]:
'''get the number of samples

create an array of arrays. So each element is the list of records we have for each class.

prior = create an array of the priors. so each prior is going to be the length of the inside array right divided by
the first calculation.

likelihood is the count of each word. so turn the array into numpy array and then take the sum across the 0-axis. then turn
that into an array. 

predict so the posterior is likelihood times the x-record + 1-likelihood times absolute value of x-1. so
if the word exists then x-1 is 0 so 1-likelihood is 0. 
'''

In [183]:
class MultinomialNB_mine(object):
    def __init__(self, alpha=1.0):
        self.alpha=alpha

    def fit(self, X,y):
        count_samples = X.shape[0]
        records_by_class = [[x for x,label in zip(X,y) if label == target] for target in np.unique(y)]
        self.class_log_prior_ = [np.log(len(records)/count_samples) for records in records_by_class]
        word_sum = np.array([np.array(records).sum(axis=0) for records in records_by_class]) + self.alpha
        count_sum = word_sum.sum(axis=1)
        self.feature_log_prob_ = np.log(word_sum / word_sum.sum(axis=1)[np.newaxis].T)
        return self
    def predict_log_proba(self,X):
        print([(self.feature_log_prob_ * x).sum(axis=1) + self.class_log_prior_ for x in X])
        return [(self.feature_log_prob_ * x).sum(axis=1) + self.class_log_prior_ for x in X]
    def predict(self,X):
        return np.argmax(self.predict_log_proba(X), axis=1)
    def feature_log_prob_(self):
        return self.feature_log_prob_
    def class_log_prior_(self):
        return self.class_log_prior_

In [198]:
import numpy as np
from sklearn.utils.extmath import safe_sparse_dot
from scipy.special import logsumexp
rng = np.random.RandomState(1)
X = rng.randint(5, size=(6, 100))
y = np.array([1, 2, 3, 4, 5, 6])
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X, y)
# print(clf.predict_log_proba(X[2:3]))
# print(clf.predict(X[2:3]))
# print(clf.feature_log_prob_*X[2:3])
jll = safe_sparse_dot(X[2:3], clf.feature_log_prob_.T) + clf.class_log_prior_
log_prob_x = logsumexp(jll, axis=1)
print(log_prob_x)
print( jll - np.atleast_2d(log_prob_x).T)
print(clf.classes_)
print(clf.classes_[np.argmax(jll, axis=1)])

[-911.99414188]
[[-68.31883031 -69.89100067   0.         -76.59265374 -85.02998333
  -83.71000399]]
[1 2 3 4 5 6]
[3]


In [202]:
clf2 = MultinomialNB_mine()
clf2.fit(X, y)
# print(clf2.feature_log_prob_==clf.feature_log_prob_)
# print(clf2.feature_log_prob_[2][3])
# print(clf.feature_log_prob_[2][3])
# print(clf.predict(X[2:3]))
# print(clf2.feature_log_prob_*X[2:3])
print(clf2.class_log_prior_)
jll = safe_sparse_dot(X[2:3], clf2.feature_log_prob_.T) + clf2.class_log_prior_
log_prob_x = logsumexp(jll, axis=1)
print (log_prob_x)
print( jll - np.atleast_2d(log_prob_x).T)
print(np.unique(y)[np.argmax(jll, axis=1)])


[-1.791759469228055, -1.791759469228055, -1.791759469228055, -1.791759469228055, -1.791759469228055, -1.791759469228055]
[-911.99414188]
[[-68.31883031 -69.89100067   0.         -76.59265374 -85.02998333
  -83.71000399]]
[3]


### GaussianNB

In [221]:
class GaussianNB_Mine(object):
    def __init__(self):
        pass

    def fit(self, X,y):
        records_by_class = [[x for x, label in zip(x,y) if label == target] for target in np.unique(y)]
        self.model = np.array([np.c_[np.mean(records,axis=0),np.std(records, axis=0)] for records in records_by_class])
        return self
    def _prob(self, x, mean, std):
        exponent = np.exp(- ((x-mean)**2 / (2* std**2)))
        return np.log(exponent / (np.sqrt(2*np.pi) * std))

    def predict_log_proba_(self, X):
        return [[sum(self._prob(i, *s) for s,i in zip(statistics, x)) for statistics in self.model] for x in X]

    def predict(self, X):
        return np.argmax(self.predict_log_proba_(X), axis=1)

    def score(self, X, y):
        return sum(self.predict(X)==y) / len(y)


In [225]:
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
Y = np.array([1, 1, 1, 2, 2, 2])
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X, Y)
print(clf.predict([[-0.8, -1]]))
print(clf.predict_log_proba([[-0.8,-1]]))

[1]
[[-5.05653266e-08 -1.67999998e+01]]


In [226]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
Y = np.array([1, 1, 1, 2, 2, 2])
clf = GaussianNB_Mine()
clf.fit(X, Y)
print(clf.predict([[-0.8, -1]]))
print(clf.predict_log_proba_([[-0.8,-1]]))

[0]
[[-3.0568998348165355, -5.006899834816537]]
